In [1]:
import mailparser
from os import listdir
from os.path import isfile, join
import pandas as pd
import email

First I will get the list of each mail directory in the dataset. There should be 150 different people in the dataset. 

In [2]:
mail_dir = '../data/maildir/'
mailboxes = listdir(mail_dir)
print(len(mailboxes))

150


Temporarily start with one person. This will be for each directoy later on. 

In [3]:
mailbox = mailboxes[0]
inbox = listdir(mail_dir + mailbox)
print(inbox)

['sent_items', 'inbox', 'discussion_threads', 'notes_inbox', 'deleted_items', 'sent', 'all_documents']


We need to start with a list, that will represent each row of the dataframe. Each item in the row is a dict of parsed email values. 

In [4]:
enron_email_list = []

Now put each email into a pandas dataframe. process_email will create a 1 row dataframe to append to the full dataframe. 

In [5]:
def process_email(email_path, employee, folder, email_name):
    mail = mailparser.parse_from_file(email_path)
    
    full_body = email.message_from_string(mail.body)
    mail_body = full_body.get_payload() 
    
    email_dict = {
                "Employee" : employee,
                "Folder" : folder,
                "Email-Name" : email_name,
                "Body" : mail_body,
    }
    #Put rest of header into the row. 
    email_dict.update(mail.headers)
    enron_email_list.append(email_dict)
    

In [6]:
#process_email(full_path, mailbox, folder[0], single_email[0])

#for mailbox in mailboxes:
    #inbox = listdir(mail_dir + mailbox)
for folder in inbox:
    path = mail_dir + mailbox + "/" + folder
    emails = listdir(path)
    for single_email in emails:
        full_path = path + "/" + single_email
        process_email(full_path, mailbox, folder, single_email)
enron_email_list_df = pd.DataFrame(enron_email_list)
enron_email_list_df

,Bcc,Body,Cc,Content-Transfer-Encoding,Content-Type,Date,Email-Name,Employee,Folder,From,...,Mime-Version,Subject,To,X-FileName,X-Folder,X-From,X-Origin,X-To,X-bcc,X-cc
0,NaN,yep. looks like i found the formula.\n\n ----...,NaN,7bit,text/plain; charset=us-ascii,"Fri, 12 Oct 2001 08:23:48 -0700 (PDT)",511.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,erichardson@sarofim.com,MLENHAR (Non-Privileged).pst,\MLENHAR (Non-Privileged)\Sent Items,"Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",Lenhart-M,'Erin Richardson <erichardson@sarofim.com>@ENRON',,
1,NaN,sorry i had to run last night. so the word is...,NaN,7bit,text/plain; charset=us-ascii,"Fri, 26 Oct 2001 07:24:43 -0700 (PDT)",414.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,hal.lenhart@apollogrp.edu,MLENHAR (Non-Privileged).pst,\MLENHAR (Non-Privileged)\Sent Items,"Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",Lenhart-M,'Hal Lenhart <Hal.Lenhart@apollogrp.edu>@ENRON',,
2,NaN,"i tried to call yesterday, but you guys were n...",NaN,7bit,text/plain; charset=us-ascii,"Mon, 14 May 2001 10:55:00 -0700 (PDT)",231.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,Re: MOTHER'S DAY,dla6138201@aol.com,"Lenhart, Matthew.pst","\Lenhart, Matthew\Lenhart, Matthew\Sent Items",Matthew Lenhart,LENHART-M,DLa6138201@aol.com <DLa6138201@aol.com@ENRON>,,
3,NaN,my friends are liars. this past weekend was a...,NaN,7bit,text/plain; charset=us-ascii,"Tue, 29 May 2001 14:12:40 -0700 (PDT)",346.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,ccates@mail.lbjs.com,"Lenhart, Matthew.pst","\Lenhart, Matthew\Lenhart, Matthew\Sent Items","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",LENHART-M,'ccates@mail.lbjs.com',,
4,NaN,if you act enthusiastic(sp?) then you'll be en...,NaN,7bit,text/plain; charset=us-ascii,"Tue, 9 Oct 2001 12:06:09 -0700 (PDT)",555.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,erichardson@sarofim.com,MLENHAR (Non-Privileged).pst,\MLENHAR (Non-Privileged)\Sent Items,"Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",Lenhart-M,'Erin Richardson <erichardson@sarofim.com>@ENRON',,
5,NaN,did you get any sleep last night with your new...,NaN,7bit,text/plain; charset=us-ascii,"Thu, 4 Oct 2001 06:48:49 -0700 (PDT)",608.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,erichardson@sarofim.com,MLENHAR (Non-Privileged).pst,\MLENHAR (Non-Privileged)\Sent Items,"Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",Lenhart-M,'Erin Richardson <erichardson@sarofim.com>@ENR...,,
6,NaN,are you kidding? why would you want a chihuah...,NaN,7bit,text/plain; charset=us-ascii,"Tue, 2 Oct 2001 13:20:29 -0700 (PDT)",621.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,erichardson@sarofim.com,MLENHAR (Non-Privileged).pst,\MLENHAR (Non-Privileged)\Sent Items,"Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",Lenhart-M,'Erin Richardson <erichardson@sarofim.com>@ENR...,,
7,NaN,i'm bout it\n\n -----Original Message-----\nFr...,NaN,7bit,text/plain; charset=us-ascii,"Wed, 23 May 2001 09:01:49 -0700 (PDT)",305.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE: Replay of MLK Holiday,"lisa.gillette@enron.com, luis.mena@enron.com","Lenhart, Matthew.pst","\Lenhart, Matthew\Lenhart, Matthew\Sent Items","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",LENHART-M,"Gillette, Lisa </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,
8,NaN,---------------------- Forwarded by Matthew Le...,NaN,7bit,text/plain; charset=us-ascii,"Mon, 16 Apr 2001 12:14:00 -0700 (PDT)",1.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,FW: granny,jay.reitmeyer@enron.com,"Lenhart, Matthew.pst","\Lenhart, Matthew\Lenhart, Matthew\Sent Items",Matthew Lenhart,LENHART-M,Jay Reitmeyer <Jay Reitmeyer/HOU/ECT@ECT>,,
9,NaN,i'm bored. tell me a story.\n\n -----Original...,NaN,7bit,text/plain; charset=us-ascii,"Fri, 5 Oct 2001 11:51:10 -0700 (PDT)",582.,lenhart-m,sent_items,matthew.lenhart@enron.com,...,1.0,RE:,erichardson@sarofim.com,MLENHAR (Non-Privileged).pst,\ML

Check and cast datatypes to appropriate types. 

In [7]:
enron_email_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5920 entries, 0 to 5919
Data columns (total 21 columns):
Bcc                          468 non-null object
Body                         5920 non-null object
Cc                           468 non-null object
Content-Transfer-Encoding    5919 non-null object
Content-Type                 5919 non-null object
Date                         5920 non-null object
Email-Name                   5920 non-null object
Employee                     5920 non-null object
Folder                       5920 non-null object
From                         5920 non-null object
Message-ID                   5920 non-null object
Mime-Version                 5919 non-null object
Subject                      5920 non-null object
To                           5833 non-null object
X-FileName                   5919 non-null object
X-Folder                     5919 non-null object
X-From                       5919 non-null object
X-Origin                     5919 non-null ob

Description of the dataset. I will now identify irrelevant headers and drop the column. 

In [8]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns',500)
enron_email_list_df.describe()

,Bcc,Body,Cc,Content-Transfer-Encoding,Content-Type,Date,Email-Name,Employee,Folder,From,Message-ID,Mime-Version,Subject,To,X-FileName,X-Folder,X-From,X-Origin,X-To,X-bcc,X-cc
count,468,5920,468,5919,5919,5920,5920,5920,5920,5920,5920,5919,5920,5833,5919,5919,5919,5919,5919,5919,5919
unique,175,3220,175,2,2,3256,2081,1,7,152,5920,1,989,426,3,16,217,2,660,1,227
top,"matthew.lenhart@enron.com, merlinm@uswest.net, pederleifkjeseth@hotmail.com, \n\tralee@rmi.net",,"matthew.lenhart@enron.com, merlinm@uswest.net, pederleifkjeseth@hotmail.com, \n\tralee@rmi.net",7bit,text/plain; charset=us-ascii,"Tue, 19 Dec 2000 05:27:00 -0800 (PST)",23.,lenhart-m,all_documents,matthew.lenhart@enron.com,<9345134.1075862004250.JavaMail.evans@thyme>,1.0,RE:,mmmarcantel@equiva.com,mlenhar.nsf,\Matthew_Lenhart_Nov2001\Notes Folders\All documents,Matthew Lenhart,Lenhart-M,"""Marcantel MM (Mitch)"" <MMMarcantel@equiva.com> @ ENRON",,
freq,15,11,15,5823,5751,8,7,5920,2056,5203,1,5919,1143,404,4626,1041,4679,3102,332,5919,5444


It appears that the column 'Mime-Version' has one value that does not seem like it would affect email summarization. It could safely be dropped. 

In [9]:
enron_email_list_df['Mime-Version'].value_counts()

1.0    5919
Name: Mime-Version, dtype: int64

In [10]:
enron_email_list_df = enron_email_list_df.drop(columns=['Mime-Version'])

In [11]:
enron_email_list_df['Content-Transfer-Encoding'].value_counts()

7bit                5823
quoted-printable    96  
Name: Content-Transfer-Encoding, dtype: int64

In [12]:
enron_email_list_df['Content-Type'].value_counts()

text/plain; charset=us-ascii          5751
text/plain; charset=ANSI_X3.4-1968    168 
Name: Content-Type, dtype: int64

In [13]:
enron_email_list_df['X-Origin'].value_counts()

Lenhart-M    3102
LENHART-M    2817
Name: X-Origin, dtype: int64